In [5]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

from pprint import pprint
import numpy as np
import descarteslabs as dl

from src.ReMasFrame import *

nasa_df = ReMasFrame()

# Choose an idx --> a landslide (or filter the geodataframe :D)
idx_test = 11031

# changes Point to Polygon
# nasa_df_polygon = nasa_df.create_box(8000000)

# Returns products that we use as a dict
products = nasa_df.get_products()

nasa_df.loc[idx_test, :]

/home/step/miniconda3/envs/projectx/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


location_description        Ayu, Ozgon, Osh, Kyrgyzstan
landslide_size                                    large
event_date                                   2017-04-29
landslide_category                  translational_slide
landslide_trigger                              downpour
fatality_count                                       24
injury_count                                        NaN
longitude                                       73.4724
latitude                                        40.8864
geometry                POINT (73.47237853 40.88639497)
Name: 11031, dtype: object

In [6]:
pprint(products['weather']['cfs'])

{'bands': ['albedo',
           'prec',
           'snow_cover',
           'snow_depth',
           'snow_water_equivalent',
           'soilmoist1',
           'soilmoist2',
           'soilmoist3',
           'tavg',
           'tmax',
           'tmin',
           'water_runoff'],
 'freq': 'daily',
 'id': 'ncep:cfsr-v2:daily:v1',
 'name': 'CFS Daily Weather',
 'res': '20km'}


In [7]:
def get_scenes(product, buffer_size, res):
    # Returns start and end date of a 4 day interval
    start_date, end_date = nasa_df.date_interval(nasa_df.event_date[idx_test], delta_minus=10)
    
    scenes, ctx = ReMasFrame.search_scenes(
        nasa_df['geometry'][idx_test].buffer(buffer_size).envelope, 
        product['id'], 
        start_date=start_date, 
        end_date=end_date, 
        limit=20
    )
    
    new_ctx = ctx.assign(resolution=res)
    
    return scenes, new_ctx, start_date, end_date

def get_composite(product, buffer_size, res):
    
    scenes, new_ctx = get_scenes(product, buffer_size, res)
    
    arr_stack = scenes.stack(product['bands'], new_ctx)
    composite = np.ma.median(arr_stack, axis=0)
    
    return (scenes, new_ctx), composite

In [8]:
products['weather']['cfs']['bands'], len(products['weather']['cfs']['bands'])

(['albedo',
  'prec',
  'snow_cover',
  'snow_depth',
  'snow_water_equivalent',
  'soilmoist1',
  'soilmoist2',
  'soilmoist3',
  'tavg',
  'tmax',
  'tmin',
  'water_runoff'],
 12)

In [9]:
idx_soil_bands = []
for idx, band in enumerate(products['weather']['cfs']['bands']):
    if 'soil' in band:
        idx_soil_bands.append(idx)
idx_soil_bands

[5, 6, 7]

In [10]:
scenes, ctx, ini, end = get_scenes(
    products['weather']['cfs'], 
    buffer_size=0.1, #  10kmx10km
    res=0.20         # 0.2 deg x pix approx 20km x pix which is resolution of the sensor
)
scene_stack = scenes.stack(products['weather']['cfs']['bands'], ctx)
scene_stack.shape

(11, 12, 2, 2)

Create new soil_moist band

In [11]:
soil_bands = scene_stack[:, idx_soil_bands, ...]

In [12]:
old_shape = list(soil_bands.shape)
old_shape[1] = 1
new_shape = tuple(old_shape)
new_soil_moist = np.sum(soil_bands, axis=1).reshape(new_shape)

scene_stack = np.concatenate((scene_stack, new_soil_moist), axis=1)

new_product = np.delete(scene_stack, idx_soil_bands, axis=1)
new_product.shape

(11, 10, 2, 2)

**Shape is (batch, channels, x, y)**

In [13]:
new_product

masked_array(
  data=[[[[65535,  1799],
          [65535, 65535]],

         [[65535,    16],
          [65535, 65535]],

         [[65535,   515],
          [65535, 65535]],

         [[65535,     2],
          [65535, 65535]],

         [[65535,     2],
          [65535, 65535]],

         [[65535, 28352],
          [65535, 65535]],

         [[65535, 28783],
          [65535, 65535]],

         [[65535, 27997],
          [65535, 65535]],

         [[65535,   233],
          [65535, 65535]],

         [[    0, 10954],
          [    0,     0]]],


        [[[65535,  1711],
          [65535, 65535]],

         [[65535,     7],
          [65535, 65535]],

         [[65535,   335],
          [65535, 65535]],

         [[65535,     0],
          [65535, 65535]],

         [[65535,     1],
          [65535, 65535]],

         [[65535, 28636],
          [65535, 65535]],

         [[65535, 29160],
          [65535, 65535]],

         [[65535, 27962],
          [65535, 65535]],

         [[6